# 🎛️ Hyperparameter Tuning: Koniec ze zgadywaniem (Grid Search)

W Machine Learningu mamy dwa rodzaje parametrów:
1.  **Parametry modelu:** To te, których model uczy się sam z danych (np. wagi w sieci neuronowej, gdzie ciąć w drzewie). Tego nie dotykamy.
2.  **Hiperparametry:** To te, które my ustawiamy PRZED treningiem (np. ile drzew w lesie, jak szybko się uczyć).

Ręczne sprawdzanie ("A może zmienię to na 5... a teraz na 10...") jest stratą czasu.

**Grid Search** działa tak:
1.  Dajesz mu listę opcji (np. Drzewa: [10, 50, 100], Głębokość: [3, 5, 10]).
2.  On tworzy **siatkę** wszystkich kombinacji ($3 \times 3 = 9$ modeli).
3.  Trenuje każdy z nich (używając Cross-Validation!).
4.  Zwraca zwycięzcę.

In [1]:
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# 1. DANE (Wina)
# Zbiór trudniejszy niż Iris (więcej klas, więcej cech)
wine = load_wine()
X = wine.data
y = wine.target

# Podział na trening i test (Testu nie dotykamy aż do samego końca!)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dane gotowe. Trening: {X_train.shape}, Test: {X_test.shape}")

Dane gotowe. Trening: (142, 13), Test: (36, 13)


## Krok 1: Model Bazowy (Na czuja)

Najpierw wytrenujmy model z domyślnymi ustawieniami, żeby mieć punkt odniesienia.

In [2]:
# Domyślny las losowy
baseline_model = RandomForestClassifier(random_state=42)
baseline_model.fit(X_train, y_train)

score = baseline_model.score(X_test, y_test)
print(f"Wynik bazowy (domyślny): {score*100:.2f}%")

Wynik bazowy (domyślny): 100.00%


## Krok 2: Definiujemy Siatkę (The Grid)

Teraz definiujemy, co chcemy przetestować.
Wybierzemy 3 najważniejsze parametry Lasu Losowego:

1.  `n_estimators`: Ile drzew? (Za mało = słabo, za dużo = wolno).
2.  `max_depth`: Jak głębokie drzewa? (Za płytkie = underfitting, za głębokie = overfitting).
3.  `min_samples_split`: Ile próbek potrzeba, żeby podzielić gałąź? (Im więcej, tym model bardziej konserwatywny).

In [3]:
# Słownik parametrów do przetestowania
param_grid = {
    'n_estimators': [50, 100, 200],      # 3 opcje
    'max_depth': [None, 10, 20],         # 3 opcje (None = bez limitu)
    'min_samples_split': [2, 5, 10]      # 3 opcje
}

# Ile modeli powstanie? 3 * 3 * 3 = 27 kombinacji.
# Każdy będzie sprawdzony 5 razy (Cross-Validation).
# Razem: 27 * 5 = 135 treningów!
print("Siatka zdefiniowana. Zaczynamy szukanie...")

Siatka zdefiniowana. Zaczynamy szukanie...


In [4]:
# Tworzymy Grid Search
# estimator = nasz model
# param_grid = nasza siatka
# cv=5 = Walidacja Krzyżowa (5-krotna)
# n_jobs=-1 = Użyj wszystkich rdzeni procesora (żeby było szybciej)
# verbose=2 = Pisz co robisz

grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           n_jobs=-1,
                           verbose=1)

# Uruchamiamy (To może chwilę potrwać)
grid_search.fit(X_train, y_train)

print("\n✅ Zakończono!")

Fitting 5 folds for each of 27 candidates, totalling 135 fits

✅ Zakończono!


## Wyniki: And the winner is...

Zobaczmy, jaka kombinacja wygrała i czy udało się pobić wynik bazowy.

In [5]:
# Najlepsze parametry
print("🏆 NAJLEPSZE PARAMETRY:")
print(grid_search.best_params_)

# Najlepszy model (automatycznie wytrenowany na całych danych treningowych)
best_model = grid_search.best_estimator_

# Sprawdźmy wynik na zbiorze testowym
final_score = best_model.score(X_test, y_test)

print("-" * 30)
print(f"Wynik bazowy:     {score*100:.2f}%")
print(f"Wynik po tuningu: {final_score*100:.2f}%")

if final_score > score:
    print("🚀 Poprawa! Warto było czekać.")
elif final_score == score:
    print("😐 Brak zmiany. Domyślne parametry były już bardzo dobre (to się zdarza).")
else:
    print("📉 Spadek? To niemożliwe na zbiorze treningowym, ale na testowym może się zdarzyć (overfitting).")

🏆 NAJLEPSZE PARAMETRY:
{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
------------------------------
Wynik bazowy:     100.00%
Wynik po tuningu: 100.00%
😐 Brak zmiany. Domyślne parametry były już bardzo dobre (to się zdarza).


## 🧠 Podsumowanie: Przekleństwo Wymiarowości

Skoro Grid Search jest taki super, dlaczego nie wrzucimy tam 10 parametrów po 10 opcji każdy?

**Tu jest haczyk (Eksplozja Kombinatoryczna).**
Policzmy:
*   Parametr A: 10 opcji
*   Parametr B: 10 opcji
*   Parametr C: 10 opcji
*   Parametr D: 10 opcji

Liczba modeli: $10 \times 10 \times 10 \times 10 = 10 000$.
Jeśli jeden model uczy się 1 minutę -> **Czekasz 7 dni.**

**Rozwiązanie:**
Gdy nie wiesz, gdzie szukać, zamiast `GridSearchCV` używa się **`RandomizedSearchCV`**.
Zamiast sprawdzać wszystko, losuje on np. 100 kombinacji z puli. Statystyka mówi, że z dużym prawdopodobieństwem trafisz w coś "wystarczająco dobrego", oszczędzając 99% czasu.